In [3]:
! pip install accelerate diffusers controlnet_aux

  Using cached controlnet_aux-0.0.9-py3-none-any.whl.metadata (6.5 kB)
  Using cached timm-0.6.7-py3-none-any.whl.metadata (33 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
Using cached controlnet_aux-0.0.9-py3-none-any.whl (282 kB)
Using cached timm-0.6.7-py3-none-any.whl (509 kB)
Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl (664.8 MB)
Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl (127.9 MB)
  Attempting uninstall: nvidia-cusolver-cu12
    Found existing installation: nvidia-cusolver-cu12 11.6.3.83
    Uninstalling nvidia-cusolver-cu12-11.6.3.83:
      Successfully uninstalled nvidia-cusolver-cu12-11.6.3.83
  Attempting uninstall: timm
    Found existing installation: timm 1.0.15
    Uninstalling timm-1.0.15:
      Successfully uninstalled timm-1.0.15


In [4]:
import torch
from controlnet_aux import CannyDetector
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from diffusers.utils import load_image,make_image_grid

/usr/local/lib/python3.11/dist-packages/controlnet_aux/mediapipe_face/mediapipe_face_common.py:7: UserWarning: The module 'mediapipe' is not installed. The package will have limited functionality. Please install it using the command: pip install 'mediapipe'
  warnings.warn(


In [5]:
controlnet = ControlNetModel.from_pretrained( "lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16, varient="fp16")

pipe = StableDiffusionControlNetPipeline.from_pretrained( "Yntec/AbsoluteReality", controlnet=controlnet, torch_dtype=torch.float16)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/920 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.45G [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/705 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


In [ ]:
pipe.load_ip_adapter("h94/IP-Adapter", subfolder="models", weight_name="ip-adapter_sd15.bin")

pipe.enable_model_cpu_offload()

In [ ]:
from google.colab import files
files.upload()

print("upload is done")

In [ ]:
img = load_image("unititled.jpg")
img

In [ ]:
ip_adap_img = load_image("ghy2.jpg")
ip_adap_img

In [ ]:
canny = CannyDetector()
canny_img = canny(img,detect_resolution=512,image_resolution=768)
canny_img

In [ ]:
prompt = """ (photorealistic:0.5), raw, clean shaved rusty,rounded hairtip, 2d cartoon style, keep the dress color exact, 512px,chrome yellow palette, smooth palette, nose will be small,starting from the eyebrow,,rounded faces,ghibli studio, pastel images, muted color,round eyes much closer to eyebrows, pronounced nosebridge, longer nose, eyes bigger, less round, maintain expression"""

pipe.set_ip_adapter_scale(0.5)

images = pipe(prompt = prompt,
              negative_prompt = "detailed face, high quality,chiseled, bright colors", height = 768, width = 768, ip_adapter_image = ip_adap_img, image = canny_img, guidance_scale = 9.5, controlnet_conditioning_scale = 0.6, num_inference_steps = 20, num_images_per_prompt = 3).images

In [ ]:

make_image_grid(images,cols=3,rows=1)